In [7]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [11]:
df_feature_vector = pd.read_parquet('data/feature_vector.parquet')
df_feature_vector.columns

Index(['id', 'title', 'description', 'costsRating', 'executionTimeRating', 'isPrepairationNeeded', 'imageLink', 'created_at', 'createdBy', 'p_tag', 'img_tag', 'b_tag', 'li_tag', 'span_tag', 'a_tag', 'u_tag', 'header_img', 'text', 'count_question_mark', 'title_len', 'text_len', 'html_len', 'species_pos', 'total_words', 'total_nomen', 'total_adjektive', 'total_numerisch', 'total_satzzeichen', 'total_konjunktion', 'nomen_quote', 'adjektive_quote', 'numerisch_quote', 'satzzeichen_quote', 'konjunktion_quote', 'event_id', 'Schnitzen', 'Backen', 'Unsere Erde', 'Pfa. Geschichte', 'Unser Bund', '1. Hilfe', 'Feuer machen', 'Versprechen', 'Karte Kompass', 'Kim-Spiele', 'Symbolik', 'Knoten', 'Küche', 'Schwarzzelte', 'Musisches', 'Haik', 'Baum', 'Sternenkunde', 'Handwerk', 'Spiele', 'Nachhaltigkeit', 'Wasser', 'Basteln', 'Geschichten', 'Unsere Sippe', 'Pflanzen', 'Forschen', 'Bewegung', 'Kreatives', 'Sommer', 'Herbst', 'Frühling', 'Winter', 'Im Haus', 'Garten', 'Videokonferenz', 'Alleine',
       '

In [10]:
df_feature_vector[['id', 'Nachhaltigkeit', 'Wasser', 'Basteln']]

,id,Nachhaltigkeit,Wasser,Basteln
0,1,False,False,False
1,2,False,False,False
2,3,False,False,False
3,4,False,False,False
4,5,False,False,False
5,6,False,False,False
6,8,True,False,False
7,11,False,False,False
8,12,False,False,False
9,33,False,False,False
